# Code to randomly insert DLAs into a sightline [v1.0]

In [50]:
# imports
from scipy import interpolate

from specdb.specdb import IgmSpec

from pyigm.surveys.dlasurvey import DLASurvey
from pyigm.fN import dla as pyi_fd
from pyigm.fN.fnmodel import FNModel

#from xastropy.xutils import xdebug as xdb

sys.path.append(os.path.abspath("../../src"))
import training_set as tset

## Random numbers

In [43]:
rstate = np.random.RandomState(1234)

## Spectra

In [3]:
igmsp = IgmSpec()

Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the catalog file
Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'HSTQSO', u'SDSS_DR7', u'KODIAQ_DR1', u'MUSoDLA', u'HD-LLS_DR1', u'2QZ', u'ESI_DLA', u'HDLA100', u'GGG', u'COS-Halos', u'HST_z2', u'COS-Dwarfs', u'XQ-100']
Database is igmspec
Created on 2016-Oct-25


## Grab the sightlines

In [2]:
reload(tset)
slines, sdict = tset.grab_sightlines()

Using the DR5 sample for the sightlines
SDSS-DR5: Loading DLA file /Users/xavier/local/Python/pyigm/pyigm/data/DLA/SDSS_DR5/dr5_alldla.fits.gz
SDSS-DR5: Loading QSOs file /Users/xavier/local/Python/pyigm/pyigm/data/DLA/SDSS_DR5/dr5_dlagz_s2n4.fits
We have 5034 sightlines for analysis
Min z = 2.23244, Median z = 2.65984, Max z = 5.19597


## Fiddling with random $z$

### z~3

In [8]:
i3 = np.argmin(np.abs(slines['ZEM']-3.))
s3 = slines[i3]
spec3l, meta = igmsp.spec_from_coord((s3['RA'], s3['DEC']), isurvey=['SDSS_DR7'])
spec3 = spec3l[0]

Your search yielded 1 match[es]
Staged 1 spectra totalling 6.4e-05 Gb
Loaded spectra


In [11]:
spec3.wvmin.value/1215.67 - 1.

2.1209319596016654

#### zlya

In [48]:
zlya = spec3.wavelength.value/1215.67 - 1
dz = np.roll(zlya,-1)-zlya
dz[-1] = dz[-2]
dz

array([ 0.0007187 ,  0.00071887,  0.00071903, ...,  0.00174   ,
        0.0017404 ,  0.0017404 ])

In [49]:
### Cut on zem and 920A rest-frame
gdz = (zlya < s3['ZEM']) & (spec3.wavelength > 910.*u.AA*(1+s3['ZEM']))

#### $\ell(z)$

In [25]:
reload(pyi_fd)
lz = pyi_fd.lX(zlya[gdz], extrap=True, calc_lz=True)
lz

array([ 0.18331483,  0.18334042,  0.18336602, ...,  0.27237858,
        0.27241322,  0.27244785])

#### Cumul

In [37]:
cum_lz = np.cumsum(lz*dz[gdz])
tot_lz = cum_lz[-1]
tot_lz
#xdb.xplot(cum_lz)

0.18932704664780059

#### Random draw

In [39]:
ndla = np.random.poisson(tot_lz, 20)
ndla

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

#### Random redshifts

In [41]:
fzdla = interpolate.interp1d(cum_lz/tot_lz, zlya[gdz],bounds_error=False,fill_value=np.min(zlya[gdz]))#

In [46]:
randz = rstate.random_sample(100)

In [47]:
zdla = fzdla(randz)
zdla

array([ 2.98564734,  2.91595431,  2.71686489,  2.95142175,  2.79594765,
        2.78945962,  2.16232933,  2.55063337,  2.4011178 ,  2.45195463,
        2.20491238,  2.82471283,  2.61489647,  2.74420171,  2.71902585,
        2.48622071,  2.55734776,  2.52803089,  2.58838005,  2.98862937,
        2.48859465,  2.13300936,  2.94525584,  2.91020805,  2.46795376,
        2.71887703,  2.47375047,  2.33323137,  2.34364447,  2.53217058,
        2.19429392,  2.73584985,  2.82060163,  2.95257784,  2.49142715,
        2.41543319,  2.48718822,  2.87657154,  2.93031168,  2.53935166,
        2.12247379,  2.16322234,  2.26281261,  2.69246489,  2.22777802,
        2.41565617,  2.17973902,  2.71164358,  2.14358104,  2.89692227,
        2.42448172,  2.82306824,  2.66294105,  2.15939305,  2.66487041,
        2.74242401,  2.30413911,  2.85346724,  2.70425838,  2.65633302,
        2.71835043,  2.76471922,  2.36052721,  2.84606566,  2.89796926,
        2.79835032,  2.76974259,  2.58098138,  2.89826008,  2.74

## Fiddling with random NHI

### Load $f(N)$ [just for the shape]

In [54]:
fN_model = FNModel.default_model()

Using P14 spline values to generate a default model
Loading: /Users/xavier/local/Python/pyigm/pyigm/data/fN/fN_spline_z24.fits.gz


### Cumulative

In [57]:
lX, cum_lX, lX_NHI = fN_model.calculate_lox(fN_model.zpivot,
    20.3,NHI_max=22.5, cumul=True)
xdb.xplot(lX_NHI, cum_lX)

### Interpolator

In [58]:
fNHI = interpolate.interp1d(cum_lX/cum_lX[-1], lX_NHI,bounds_error=False,fill_value=lX_NHI[0])#

In [59]:
randNHI = rstate.random_sample(100)

In [60]:
dla_NHI = fNHI(randNHI)
dla_NHI

array([ 20.48331731,  21.67512512,  20.59492666,  20.72369414,
        20.67841503,  20.4226547 ,  20.40168945,  20.654418  ,
        21.73114995,  20.56439854,  20.88243825,  20.40832829,
        20.34198151,  20.39645882,  20.39900071,  20.4817214 ,
        20.68728349,  20.96605553,  21.16749841,  20.79019633,
        21.10583448,  20.74783509,  20.8566711 ,  20.68772767,
        21.18949896,  20.33081183,  20.98149793,  20.38846215,
        20.33469602,  20.33487522,  20.33063393,  20.66801997,
        20.57315366,  22.24008286,  20.36533816,  20.51346323,
        21.98394868,  20.57005345,  21.17476473,  20.71862699,
        21.41103761,  21.17982791,  20.97538906,  20.31855475,
        21.46239687,  20.67137241,  20.42941226,  21.15508122,
        20.44042832,  20.38430537,  20.89732151,  21.53947189,
        20.6861296 ,  20.60572458,  20.31279553,  20.75622788,
        21.47579246,  21.49400243,  21.31255748,  20.41408149,
        20.77435049,  20.57955718,  20.54032425,  20.32

In [63]:
rstate.rand?